In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2019 Intel Corporation. All Rights Reserved.
# Python 2/3 compatibility
from __future__ import print_function

# First import the library
import pyrealsense2 as rs

# Import OpenCV and numpy
import cv2
import numpy as np
from math import tan, pi
import ipywidgets
import time
wid = ipywidgets.Image()

# Set up a mutex to share data between threads 设置互斥体以在线程之间共享数据
from threading import Lock
frame_mutex = Lock()
frame_data = {"left"  : None,
              "right" : None,
              "timestamp_ms" : None
              }

"""
这个回调是在单独的线程上调用的，
所以我们必须使用互斥锁来确保数据正确同步。
我们还应该注意不要在这个线程上做太多工作，
以避免回调队列中的数据备份。
"""
def callback(frame):
    global frame_data
    if frame.is_frameset():
        frameset = frame.as_frameset()
        f1 = frameset.get_fisheye_frame(1).as_video_frame()
        f2 = frameset.get_fisheye_frame(2).as_video_frame()
        left_data = np.asanyarray(f1.get_data())
        right_data = np.asanyarray(f2.get_data())
        ts = frameset.get_timestamp()
        frame_mutex.acquire()
        frame_data["left"] = left_data
        frame_data["right"] = right_data
        frame_data["timestamp_ms"] = ts
        frame_mutex.release()

# 声明RealSense管道，封装实际设备和传感器
pipe = rs.pipeline()

# 构建配置对象并流式传输所有内容
cfg = rs.config()

# 使用我们的回调开始流式传输
pipe.start(cfg, callback)

try:
    WINDOW_TITLE = 'Realsense'
    cv2.namedWindow(WINDOW_TITLE, cv2.WINDOW_NORMAL)
   
    # 检索两个相机的流和固有属性
    profiles = pipe.get_active_profile()
    streams = {"left"  : profiles.get_stream(rs.stream.fisheye, 1).as_video_stream_profile(),
               "right" : profiles.get_stream(rs.stream.fisheye, 2).as_video_stream_profile()}
    intrinsics = {"left"  : streams["left"].get_intrinsics(),
                  "right" : streams["right"].get_intrinsics()}

    # 打印有关两个摄像头的信息
    print("Left camera:",  intrinsics["left"])
    print("Right camera:", intrinsics["right"])
    frame_width=848
    frame_height=800
    # 创建保存视频的对象，设置编码格式，帧率，图像的宽高等
    color_fisheye_image_right_out = cv2.VideoWriter('fisheye_image/color_fisheye_image_right_out.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))  # 保存视频
    color_fisheye_image_lift_out = cv2.VideoWriter('fisheye_image/color_fisheye_image_lift_out.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))  # 保存视频
    color_fisheye_image_all_out =  cv2.VideoWriter('fisheye_image/color_fisheye_image_all_out.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width*2, frame_height))  # 保存视频

    # 设置OpenCV窗口以可视化结果
    display(wid)

    T1 = time.time()
    while True:
        # 检查摄像头是否已获取任何帧
        frame_mutex.acquire()
        valid = frame_data["timestamp_ms"] is not None
        frame_mutex.release()

        # 如果帧已准备好处理
        if valid:
            
            color_fisheye_image_left = cv2.cvtColor(frame_data["left"],cv2.COLOR_GRAY2RGB)
            color_fisheye_image_right = cv2.cvtColor(frame_data["right"],cv2.COLOR_GRAY2RGB)
             # 将每一帧图像写入到输出文件中
            color_fisheye_image_right_out.write(color_fisheye_image_right)  #视频写入
            color_fisheye_image_lift_out.write(color_fisheye_image_left)
            color_fisheye_image_all_out.write(np.hstack((color_fisheye_image_left,color_fisheye_image_right)))
            
            cv2.imshow(WINDOW_TITLE, np.hstack((color_fisheye_image_left,color_fisheye_image_right)))
        key = cv2.waitKey(1)
        if key == ord('q') or cv2.getWindowProperty(WINDOW_TITLE, cv2.WND_PROP_VISIBLE) < 1:
            color_fisheye_image_right_out.release()  #资源释放
            color_fisheye_image_lift_out.release()
            color_fisheye_image_all_out.release()
            cv2.destroyAllWindows()
            break
          
finally:
    
    pipe.stop()


Left camera: [ 848x800  p[422.208 403.578]  f[285.949 285.99]  Kannala Brandt4 [-0.00449127 0.0374638 -0.0352701 0.0054871 0] ]
Right camera: [ 848x800  p[426.173 403.557]  f[285.737 285.935]  Kannala Brandt4 [-0.00782555 0.0477229 -0.0446198 0.00828991 0] ]


Image(value=b'')